In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# you can get this from here:
# https://github.com/cocodataset/cocoapi
import sys
sys.path.append('/home/dan/work/cocoapi/PythonAPI/')
from pycocotools.coco import COCO

In [ ]:
ANNOTATIONS_FILE = '/mnt/datasets/dan/moda/annotations/modanet2018_instances_train.json'
IMAGES_DIR = '/mnt/datasets/dan/moda/images/train/'
DOWNLOAD_METADATA = 'train_metadata.csv'

In [ ]:
coco = COCO(ANNOTATIONS_FILE)

# Get categories

In [ ]:
categories = coco.loadCats(coco.getCatIds())
names = [cat['name'] for cat in categories]
print('COCO categories: \n{}'.format(' '.join(names)))
print('\nnumber of labels:', len(names))

# Create label encoding

In [ ]:
# number of row - integer encoding
if not os.path.exists('modanet_labels.txt'):
    with open('modanet_labels.txt', 'w') as f:
        for n in names:
            f.write(n + '\n')

# Load a list of available images

In [ ]:
available_ids = []
for T in pd.read_csv(DOWNLOAD_METADATA).itertuples():
    if T.is_ok:
        available_ids.append(T.id)

# Show a random image

In [ ]:
imgIds = coco.getImgIds()

In [ ]:
i = np.random.randint(0, len(imgIds))
metadata = coco.loadImgs(imgIds[i])[0]

if metadata['id'] in available_ids:
    
    image_name = str(metadata['id']) + '.jpg'
    image = cv2.imread(os.path.join(IMAGES_DIR, image_name))
    image = image[:, :, [2, 1, 0]]  # to RGB

    annIds = coco.getAnnIds(imgIds=metadata['id'])
    annotations = coco.loadAnns(annIds)

    plt.imshow(image)
    plt.axis('off')
    coco.showAnns(annotations)

# Create a train-val split

In [ ]:
metadata = []

for i in range(len(imgIds)):
    image_metadata = coco.loadImgs(imgIds[i])[0]
    print(image_metadata['id'])
    if image_metadata['id'] in available_ids:
        is_train = np.random.rand() > 0.1
        metadata.append((imgIds[i], is_train))
        
metadata = pd.DataFrame(metadata, columns=['image_id', 'is_train'])
metadata.to_csv('trainval_split.csv', index=False)

In [ ]:
len(metadata) - metadata.is_train.sum()